In [1]:
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd

In [62]:
data_path = Path("/media/nvme1/pbecg-data/fda/2018/R^ECG^F_37^61_1152087555^015^_20180131115810_20180426194405.xml")
tree = ET.parse(data_path)
root = tree.getroot()

In [87]:
# Extract the sequence id and other sequence related metadata
namespaces = {"ns": "urn:hl7-org:v3"}
seq_id = root.find(".//ns:id", namespaces).attrib["root"]
subject_id = root.find(".//ns:trialSubject/ns:id", namespaces).attrib["extension"]
acq_time = root.find(".//ns:effectiveTime/ns:low", namespaces).attrib["value"]
# Extract the ECG signal, unit, and scale
ecg_data = []
for component in root.findall(".//ns:sequenceSet/ns:component", namespaces)[1:13]:
    sequence = component.find(".//ns:sequence", namespaces)
    if sequence is not None:
        lead = sequence.find(".//ns:code", namespaces).attrib["code"]
        scale_element = sequence.find(".//ns:scale", namespaces)
        unit = scale_element.attrib["unit"]
        scale = float(scale_element.attrib["value"])
        signal = [int(x)*scale for x in sequence.find(".//ns:digits", namespaces).text.strip().split()]
        print(signal)
    
    ecg_data.append({
        "lead": lead,
        "unit": unit,
        "signal": signal
    })

df = pd.DataFrame(ecg_data)

[-6.25, -6.25, -12.5, -18.75, -25.0, -31.25, -27.5, -31.25, -10.0, 12.5, 21.25, 31.25, 21.25, 25.0, 25.0, 18.75, 31.25, 31.25, 25.0, 31.25, 28.75, 25.0, 10.0, 12.5, 6.25, 6.25, 15.0, 18.75, 25.0, 25.0, 22.5, 25.0, 28.75, 31.25, 33.75, 31.25, 37.5, 50.0, 52.5, 56.25, 53.75, 56.25, 53.75, 50.0, 62.5, 56.25, 50.0, 50.0, 52.5, 56.25, 52.5, 56.25, 60.0, 62.5, 53.75, 50.0, 56.25, 56.25, 56.25, 50.0, 50.0, 50.0, 47.5, 43.75, 46.25, 50.0, 47.5, 50.0, 58.75, 68.75, 77.5, 81.25, 81.25, 81.25, 78.75, 75.0, 72.5, 68.75, 77.5, 81.25, 78.75, 81.25, 81.25, 75.0, 77.5, 81.25, 77.5, 75.0, 87.5, 81.25, 75.0, 75.0, 85.0, 75.0, 78.75, 81.25, 75.0, 75.0, 77.5, 75.0, 68.75, 62.5, 60.0, 56.25, 58.75, 62.5, 56.25, 56.25, 68.75, 75.0, 68.75, 62.5, 53.75, 43.75, 37.5, 31.25, 31.25, 37.5, 37.5, 31.25, 22.5, 12.5, 6.25, 6.25, 6.25, 6.25, 0.0, 0.0, 8.75, 12.5, 21.25, 25.0, 25.0, 18.75, 18.75, 18.75, 21.25, 25.0, 18.75, 12.5, 0.0, 0.0, -6.25, -6.25, 2.5, 6.25, 18.75, 25.0, 16.25, 18.75, 27.5, 25.0, 12.5, 12.5, 6.25

In [86]:
df

,lead,unit,signal
0,MDC_ECG_LEAD_I,uV,"[-6.25, -6.25, -12.5, -18.75, -25.0, -31.25, -..."
1,MDC_ECG_LEAD_II,uV,"[-12.5, 0.0, 0.0, -6.25, -2.5, -12.5, -6.25, -..."
2,MDC_ECG_LEAD_III,uV,"[-6.25, 6.25, 12.5, 12.5, 22.5, 18.75, 21.25, ..."
3,MDC_ECG_LEAD_aVR,uV,"[10.0, 3.75, 6.25, 12.5, 13.75, 22.5, 17.5, 18..."
4,MDC_ECG_LEAD_aVL,uV,"[0.0, -6.25, -12.5, -16.25, -23.75, -25.0, -25..."
5,MDC_ECG_LEAD_aVF,uV,"[-10.0, 2.5, 6.25, 2.5, 10.0, 2.5, 7.5, 8.75, ..."
6,MDC_ECG_LEAD_V1,uV,"[-18.75, -12.5, -6.25, 0.0, -6.25, -12.5, -6.2..."
7,MDC_ECG_LEAD_V2,uV,"[-37.5, -37.5, -37.5, -37.5, -37.5, -37.5, -37..."
8,MDC_ECG_LEAD_V3,uV,"[-112.5, -100.0, -100.0, -100.0, -100.0, -100...."
9,MDC_ECG_LEAD_V4,uV,"[-118.75, -112.5, -112.5, -112.5, -118.75, -11..."
